In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
def SoupFromURL(url, suppressOutput=True):

    if not suppressOutput:
        print(url)
    try:
        r = requests.get(url)
    except:
        return None

    return BeautifulSoup(r.text, "html5lib")

def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

In [3]:
def nba_player_dict(url):
    player_page = SoupFromURL(str(url))
    a = player_page.find_all('div', attrs={'itemtype':'https://schema.org/Person'})[0]
    player_dict = {}
    try:
        b_soup = BeautifulSoup(str(a),"html5lib")
        b = b_soup.text.split('\n')
        b = list(filter(None, b))
        age_index = b.index('  Born:')
        age_born = b[age_index:]
        age_year = age_born[3]
        age_year = int(age_year.replace('            ', ''))
        age = 2017 - age_year
        player_dict['age'] = age
    except:
        player_dict['age'] = None
    
    a_soup = BeautifulSoup(str(a),"html5lib")
    a = a_soup.text.split('\n')
    a = list(filter(None, a))
    to_remove = ['  ','    ','  ▪','\t','  Position:','  Shoots:','  Born:','  us','  in']
    for remove in to_remove:
        a=remove_values_from_list(a,remove)
    
    starts_with_removal = ['(','      ','            ','    in','    More bio','  NBA Debut','  ABA Debut','Relatives:','  Pronunciation:']
    for removal in starts_with_removal:
        a=[x for x in a if not x.startswith(removal)]
    
    if a[2] == '  Twitter:':
        del a[2:4]
    else:
        a
    
    if a[-2] == '  Experience:':
        a.pop()
        a.pop()
    else:
        a
    
    if a[5] == '  Died:':
        del a[5:7]
    else:
        a
        
    try:
        name = a[0]
        player_dict['player_name'] = name.replace("\t","")
    except:
        player_dict['player_name'] = None
        
    try:
        position = a[2]
        player_dict['position'] = position.replace('  ','')
    except:
        player_dict['position'] = None
        
    try:
        shooting_hand = a[3]
        player_dict['shooting_hand'] = shooting_hand.replace('  ','')
    except:
        player_dict['shooting_hand'] = None
    
    try:
        body = a[4].split(',')
        ft = int(body[0][0])*12
        if len(body[0]) == 3:
            inch = int(body[0][-1])
        elif len(body[0]) == 4:
            inch = int(body[0][2:4])
        player_dict['height_inches'] = ft+inch
    
        w = body[1].replace(u'\xa0', u'')
        player_dict['weight_lbs'] = int(w[0:3])
    except:
        player_dict['height_inches']=None
        player_dict['weight_lbs']=None
    
    if a[5] == '  College:':
        college = a[6]
        player_dict['college'] = college.replace('  ','')
    elif a[6] == '  College:':
        college = a[7]
        player_dict['college'] = college.replace('  ','')
    else:
        player_dict['college'] = 'no_college'
    
    if a[5].startswith('  Team'):
        player_dict['current_team'] = a[5][8:]
    else:
        player_dict['current_team'] = 'no_current_team'
    
    if a[-2] == '  Draft:':
        draft = a[-1].split(',')
        player_dict['draft_year'] = int(draft[-1][1:5])
        player_dict['draft_position'] = int(re.sub('\D','',draft[2]))
    elif a[-3] == '  Draft:':
        draft = a[-2].split(',')
        player_dict['draft_year'] = int(draft[-1][1:5])
        draft_position = int(re.sub('\D','',draft[2]))
    else:
        draft='not_drafted'
        player_dict['draft_year']='not_drafted'
        player_dict['draft_position']='not_drafted'
    
    try:
        all_s = player_page.find_all('ul', attrs={'id':'bling'})
        all_soup = BeautifulSoup(str(all_s),"html5lib")
        all_s = all_soup.text.split('\n')
    
        if any('All Star' in s for s in all_s):
            player_dict['all_star']=1
        else:
            player_dict['all_star']=0
    except:
        player_dict['all_star']=0
    
    stat_list = []
    try:
        p = player_page.find_all('div',attrs={'class':'stats_pullout'})
        p_soup = BeautifulSoup(str(p),"html5lib")
        stat_career = p_soup.text.split('\n')[1::4][2:]
        for s in stat_career:
            try:
                if s== str(""):
                    stat_list.append(None)
                else:
                    stat_list.append(float(s))
            except:
                stat_list.append(None)
    except:
        stat_list.extend([None,None,None,None,None,None,None,None,None,None])
    
    try:
        player_dict['per'] = stat_list[-2]
    except:
        player_dict['per'] = None
    
    try:
        player_dict['ws'] = stat_list[-1]
    except:
        player_dict['ws'] = None
        
    additional_stats = player_page.find_all('td', attrs={'class':'right'})
    additional_soup = BeautifulSoup(str(additional_stats),'html5lib')
    additional_soup = additional_soup.text.split(',')
    new_add_soup = [additional_soup[i:i+25] for i in range(0, len(additional_soup), 25)]
    
    try:
        empty_list = [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
        target_index = new_add_soup.index(empty_list)
        stats_lists = new_add_soup[:target_index]
    except:
        stats_lists = new_add_soup
    
    career_per_game = stats_lists[-1]
    career_per_game = [item.replace(']', '') for item in career_per_game]
    
    stats_columns = [
        'g',
        'gs',
        'mp',
        'fg',
        'fga',
        'fg_perc',
        '3p',
        '3pa',
        '3p_perc',
        '2p',
        '2pa',
        '2p_perc',
        'efg_perc',
        'ft',
        'fta',
        'ft_perc',
        'orb',
        'drb',
        'trb',
        'ast',
        'stl',
        'blk',
        'tov',
        'pf',
        'pts'
    ]
    
    number_list = list(range(25))
    zipped = zip(stats_columns,number_list)
    
    for column,number in zipped:
        try:
            player_dict[column]=float(career_per_game[number])
        except:
            player_dict[column]=None
    
    additional_stats_2 = player_page.find_all('td', attrs={'class':'left'})
    additional_soup_2 = BeautifulSoup(str(additional_stats_2),'html5lib')
    additional_soup_2 = additional_soup_2.text.split(',')

    additional_soup_2 = remove_values_from_list(additional_soup_2,' NBA')
    try:
        target_ind = additional_soup_2.index(' ')
        team_id = additional_soup_2[:target_ind]
    except:
        team_id = additional_soup_2
    try:
        teams = []
        for i in team_id:
            j = i.replace(' ','')
            teams.append(j)
    
        salary_team = teams[-1]
        salary_team = re.sub('[^0-9a-zA-Z]+', '', salary_team)
        player_dict['salary_team'] = salary_team
        season_count = len(teams)
        player_dict['season_count'] = season_count
    except:
        player_dict['salary_team'] = None
        player_dict['season_count'] = None
    
    try:
        salary_stats = player_page.find_all('div',{'id':'all_all_salaries'})
        salary_str = str(salary_stats)
        salary_str = salary_str.split('\n')
        salary_str = [k for k in salary_str if 'salary' in k]
    
        salary_select = salary_str[-2][-22:-1]
        player_dict['recent_salary'] = float(re.sub('\D','',salary_select))
    
        salary_year = salary_str[-2][50:76]
        salary_year = re.sub('\D','',salary_year)
        player_dict['salary_year'] = int(salary_year[0:4])
    except:
        player_dict['recent_salary'] = 'not_listed'
        player_dict['salary_year'] = 'not_listed'
    
    return player_dict

In [4]:
nba_player_dict('https://www.basketball-reference.com/players/p/paulch01.html')

{'2p': 5.4,
 '2p_perc': 0.504,
 '2pa': 10.7,
 '3p': 1.2,
 '3p_perc': 0.37,
 '3pa': 3.2,
 'age': 32,
 'all_star': 1,
 'ast': 9.9,
 'blk': 0.1,
 'college': 'Wake Forest University',
 'current_team': 'Houston Rockets',
 'draft_position': 4,
 'draft_year': 2005,
 'drb': 3.7,
 'efg_perc': 0.516,
 'fg': 6.6,
 'fg_perc': 0.473,
 'fga': 13.9,
 'ft': 4.3,
 'ft_perc': 0.866,
 'fta': 5.0,
 'g': 834.0,
 'gs': 834.0,
 'height_inches': 72,
 'mp': 35.5,
 'orb': 0.7,
 'per': 25.7,
 'pf': 2.5,
 'player_name': 'Chris Paul',
 'position': 'Point Guard',
 'pts': 18.7,
 'recent_salary': 21468696.0,
 'salary_team': 'LAC',
 'salary_year': 2015,
 'season_count': 12,
 'shooting_hand': 'Right',
 'stl': 2.3,
 'tov': 2.4,
 'trb': 4.4,
 'weight_lbs': 175,
 'ws': 154.6}

In [5]:
def generate_nba_player_df(first_letter):
    index_url_link = 'https://www.basketball-reference.com/players/{}/'.format(first_letter)
    
    player_urls = []
    index_page = SoupFromURL(index_url_link)
    index_names = index_page.find_all('tr')
    index_soup = BeautifulSoup(str(index_names),"html5lib")
    links = index_soup('a', href=True)

    for l in links:
        try:
            player_urls.append('https://www.basketball-reference.com' + l.attrs['href'])
        except:
            pass
    
    player_urls = [x for x in player_urls if not 'colleges' in x]
    player_urls = [x for x in player_urls if not 'birthdays' in x]
    
    player_stats_list=[]
    for url in player_urls:
        try:
            player_stats_list.append(nba_player_dict(url))
            print(url)
        except:
            pass
    
    df = pd.DataFrame(player_stats_list)
    df = df[[
        'player_name',
        'position',
        'shooting_hand',
        'height_inches',
        'weight_lbs',
        'college',
        'draft_year',
        'draft_position',
        'season_count',
        'age',
        'current_team',
        'g',
        'gs',
        'mp',
        'pts',
        'fg',
        'fga',
        'fg_perc',
        '3p',
        '3pa',
        '3p_perc',
        '2p',
        '2pa',
        '2p_perc',
        'efg_perc',
        'ft',
        'fta',
        'ft_perc',
        'orb',
        'drb',
        'trb',
        'ast',
        'stl',
        'blk',
        'tov',
        'pf',
        'all_star',
        'per',
        'ws',
        'recent_salary',
        'salary_year',
        'salary_team'
    ]]
    return df


In [6]:
generate_nba_player_df('y')

https://www.basketball-reference.com/players/y/yarbrvi01.html
https://www.basketball-reference.com/players/y/yardlge01.html
https://www.basketball-reference.com/players/y/yatesba01.html
https://www.basketball-reference.com/players/y/yateswa01.html
https://www.basketball-reference.com/players/y/yelvech01.html
https://www.basketball-reference.com/players/y/yonakri01.html
https://www.basketball-reference.com/players/y/youngda01.html
https://www.basketball-reference.com/players/y/youngja01.html
https://www.basketball-reference.com/players/y/youngjo01.html
https://www.basketball-reference.com/players/y/youngko01.html
https://www.basketball-reference.com/players/y/youngmi01.html
https://www.basketball-reference.com/players/y/youngni01.html
https://www.basketball-reference.com/players/y/youngpe01.html
https://www.basketball-reference.com/players/y/youngsa01.html
https://www.basketball-reference.com/players/y/youngth01.html
https://www.basketball-reference.com/players/y/youngti01.html
https://

,player_name,position,shooting_hand,height_inches,weight_lbs,college,draft_year,draft_position,season_count,age,...,stl,blk,tov,pf,all_star,per,ws,recent_salary,salary_year,salary_team
0,Vincent Yarbrough,Shooting Guard,Right,79,210,University of Tennessee,2002,33.0,1,36,...,1.0,0.6,1.4,2.5,0,9.5,1.0,8.34946e+06,2002,DEN
1,George Yardley,Small Forward,Right,77,190,Stanford University,1950,NaN,9,89,...,NaN,NaN,NaN,3.0,1,20.5,58.5,not_listed,not_listed,SYR
2,Barry Yates,Power Forward,Right,79,215,University of Maryland,1971,131.0,1,71,...,NaN,NaN,NaN,0.6,0,10.7,-0.2,not_listed,not_listed,PHI
3,Wayne Yates,Center,Right,80,235,University of Memphis,1961,5.0,1,80,...,NaN,NaN,NaN,1.9,0,4.1,-0.5,not_listed,not_listed,LAL
4,Charlie Yelverton,Shooting Guard,Right,74,190,Fordham University,1971,25.0,1,69,...,NaN,NaN,NaN,2.1,0,10.3,-0.5,not_listed,not_listed,POR
5,Rich Yonakor,Power Forward,Left,81,220,University of North Carolina,1980,61.0,1,59,...,0.1,0.2,0.2,0.7,0,20.4,0.3,not_listed,not_listed,SAS
6,Danny Young,Point Guard,Right,75,175,Wake Forest University,1984,39.0,12,55,...,0.9,0.1,0.8,0.9,0,12.2,20.2,150000,1994,MIL
7,James Young,Shooting Guard,Left,78,215,University of Kentucky,2014,17.0,3,22,...,0.3,0.1,0.2,0.6,0,7.6,0.7,1.8252e+06,2016,BOS
8,Joe Young,Point Guard,Right,74,180,University of Oregon,2015,43.0,2,25,...,0.3,0.0,0.5,0.5,0,10.3,-0.1,1.05234e+06,2016,IND
9,Korleone Young,Small Forward,Right,79,213,no_college,1998,40.0,1,39,...,0.0,0.0,0.3,1.0,0,31.3,0.1,287500,1998,DET


In [7]:
def nba_2016_player_dict(url):
    player_page = SoupFromURL(str(url))
    player_dict = {}
    stat_list = []
    new = []
    try:
        p = player_page.find_all('div',attrs={'class':'stats_pullout'})
        p_soup = BeautifulSoup(str(p),"html5lib")
        stat_career = p_soup.text.split('\n')[0::4][2:] #change this to 0::4
        for item in stat_career:
            new.append(float(''.join(i for i in item if i in '1234567890.')))
        for s in new:
            try:
                if s== str(""):
                    stat_list.append(None)
                else:
                    stat_list.append(float(s))
            except:
                stat_list.append(None)
    except:
        stat_list.extend([None,None,None,None,None,None,None,None,None,None])    

    if stat_list == [None, None, None, None, None, None, None, None, None, None]:
        return stat_list[100]
    
    else:
        try:
            player_dict['per'] = stat_list[-2]
        except:
            player_dict['per'] = None
        
        try:
            player_dict['ws'] = stat_list[-1]
        except:
            player_dict['ws'] = None
            
        a = player_page.find_all('div', attrs={'itemtype':'https://schema.org/Person'})[0]
        try:
            b_soup = BeautifulSoup(str(a),"html5lib")
            b = b_soup.text.split('\n')
            b = list(filter(None, b))
            age_index = b.index('  Born:')
            age_born = b[age_index:]
            age_year = age_born[3]
            age_year = int(age_year.replace('            ', ''))
            age = 2017 - age_year
            player_dict['age'] = age
        except:
            player_dict['age'] = None
        
        a_soup = BeautifulSoup(str(a),"html5lib")
        a = a_soup.text.split('\n')
        a = list(filter(None, a))
        to_remove = ['  ','    ','  ▪','\t','  Position:','  Shoots:','  Born:','  us','  in']
        for remove in to_remove:
            a=remove_values_from_list(a,remove)
        
        starts_with_removal = ['(','      ','            ','    in','    More bio','  NBA Debut','  ABA Debut','Relatives:','  Pronunciation:']
        for removal in starts_with_removal:
            a=[x for x in a if not x.startswith(removal)]
        
        if a[2] == '  Twitter:':
            del a[2:4]
        else:
            a
        
        if a[-2] == '  Experience:':
            a.pop()
            a.pop()
        else:
            a
        
        if a[5] == '  Died:':
            del a[5:7]
        else:
            a
            
        try:
            name = a[0]
            player_dict['player_name'] = name.replace("\t","")
        except:
            player_dict['player_name'] = None
            
        try:
            position = a[2]
            player_dict['position'] = position.replace('  ','')
        except:
            player_dict['position'] = None
            
        try:
            shooting_hand = a[3]
            player_dict['shooting_hand'] = shooting_hand.replace('  ','')
        except:
            player_dict['shooting_hand'] = None
        
        try:
            body = a[4].split(',')
            ft = int(body[0][0])*12
            if len(body[0]) == 3:
                inch = int(body[0][-1])
            elif len(body[0]) == 4:
                inch = int(body[0][2:4])
            player_dict['height_inches'] = ft+inch
        
            w = body[1].replace(u'\xa0', u'')
            player_dict['weight_lbs'] = int(w[0:3])
        except:
            player_dict['height_inches']=None
            player_dict['weight_lbs']=None
        
        if a[5] == '  College:':
            college = a[6]
            player_dict['college'] = college.replace('  ','')
        elif a[6] == '  College:':
            college = a[7]
            player_dict['college'] = college.replace('  ','')
        else:
            player_dict['college'] = 'no_college'
        
        if a[5].startswith('  Team'):
            player_dict['current_team'] = a[5][8:]
        else:
            player_dict['current_team'] = 'no_current_team'
        
        if a[-2] == '  Draft:':
            draft = a[-1].split(',')
            player_dict['draft_year'] = int(draft[-1][1:5])
            player_dict['draft_position'] = int(re.sub('\D','',draft[2]))
        elif a[-3] == '  Draft:':
            draft = a[-2].split(',')
            player_dict['draft_year'] = int(draft[-1][1:5])
            draft_position = int(re.sub('\D','',draft[2]))
        else:
            draft='not_drafted'
            player_dict['draft_year']='not_drafted'
            player_dict['draft_position']='not_drafted'
        
        try:
            all_s = player_page.find_all('ul', attrs={'id':'bling'})
            all_soup = BeautifulSoup(str(all_s),"html5lib")
            all_s = all_soup.text.split('\n')
        
            if any('All Star' in s for s in all_s):
                player_dict['all_star']=1
            else:
                player_dict['all_star']=0
        except:
            player_dict['all_star']=0
        
            
        additional_stats = player_page.find_all('td', attrs={'class':'right'})
        additional_soup = BeautifulSoup(str(additional_stats),'html5lib')
        additional_soup = additional_soup.text.split(',')
        new_add_soup = [additional_soup[i:i+25] for i in range(0, len(additional_soup), 25)]
        
        try:
            empty_list = [' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']
            target_index = new_add_soup.index(empty_list)
            stats_lists = new_add_soup[:target_index]
        except:
            stats_lists = new_add_soup
        
        career_per_game = stats_lists[-2]
        career_per_game = [item.replace(']', '') for item in career_per_game]
        
        stats_columns = [
            'g',
            'gs',
            'mp',
            'fg',
            'fga',
            'fg_perc',
            '3p',
            '3pa',
            '3p_perc',
            '2p',
            '2pa',
            '2p_perc',
            'efg_perc',
            'ft',
            'fta',
            'ft_perc',
            'orb',
            'drb',
            'trb',
            'ast',
            'stl',
            'blk',
            'tov',
            'pf',
            'pts'
        ]
        
        number_list = list(range(25))
        zipped = zip(stats_columns,number_list)
        
        for column,number in zipped:
            try:
                player_dict[column]=float(career_per_game[number])
            except:
                player_dict[column]=None
        
        additional_stats_2 = player_page.find_all('td', attrs={'class':'left'})
        additional_soup_2 = BeautifulSoup(str(additional_stats_2),'html5lib')
        additional_soup_2 = additional_soup_2.text.split(',')

        additional_soup_2 = remove_values_from_list(additional_soup_2,' NBA')
        try:
            target_ind = additional_soup_2.index(' ')
            team_id = additional_soup_2[:target_ind]
        except:
            team_id = additional_soup_2
        try:
            teams = []
            for i in team_id:
                j = i.replace(' ','')
                teams.append(j)
            
            salary_team = teams[-1]
            salary_team = re.sub('[^0-9a-zA-Z]+', '', salary_team)
            player_dict['salary_team'] = salary_team
            season_count = len(teams)
            player_dict['season_count'] = season_count
    
        except:
            player_dict['salary_team'] = None
            player_dict['season_count'] = None
        
        try:
            salary_stats = player_page.find_all('div',{'id':'all_all_salaries'})
            salary_str = str(salary_stats)
            salary_str = salary_str.split('\n')
            salary_str = [k for k in salary_str if 'salary' in k]
        
            salary_select = salary_str[-2][-22:-1]
            player_dict['recent_salary'] = float(re.sub('\D','',salary_select))
        
            salary_year = salary_str[-2][50:76]
            salary_year = re.sub('\D','',salary_year)
            player_dict['salary_year'] = int(salary_year[0:4])
        except:
            player_dict['recent_salary'] = 'not_listed'
            player_dict['salary_year'] = 'not_listed'
        
        return player_dict

In [8]:
nba_2016_player_dict('https://www.basketball-reference.com/players/t/thomais02.html')

{'2p': 5.8,
 '2p_perc': 0.528,
 '2pa': 10.9,
 '3p': 3.2,
 '3p_perc': 0.379,
 '3pa': 8.5,
 'age': 28,
 'all_star': 1,
 'ast': 5.9,
 'blk': 0.2,
 'college': 'University of Washington',
 'current_team': 'Cleveland Cavaliers',
 'draft_position': 60,
 'draft_year': 2011,
 'drb': 2.1,
 'efg_perc': 0.546,
 'fg': 9.0,
 'fg_perc': 0.463,
 'fga': 19.4,
 'ft': 7.8,
 'ft_perc': 0.909,
 'fta': 8.5,
 'g': 76.0,
 'gs': 76.0,
 'height_inches': 69,
 'mp': 33.8,
 'orb': 0.6,
 'per': 26.5,
 'pf': 2.2,
 'player_name': 'Isaiah Thomas',
 'position': 'Point Guard',
 'pts': 28.9,
 'recent_salary': 6587132.0,
 'salary_team': 'BOS',
 'salary_year': 2016,
 'season_count': 8,
 'shooting_hand': 'Left',
 'stl': 0.9,
 'tov': 2.8,
 'trb': 2.7,
 'weight_lbs': 185,
 'ws': 12.5}

In [9]:
def generate_2016_nba_player_df(first_letter):
    index_url_link = 'https://www.basketball-reference.com/players/{}/'.format(first_letter)
    
    player_urls = []
    index_page = SoupFromURL(index_url_link)
    index_names = index_page.find_all('tr')
    index_soup = BeautifulSoup(str(index_names),"html5lib")
    links = index_soup('a', href=True)

    for l in links:
        try:
            player_urls.append('https://www.basketball-reference.com' + l.attrs['href'])
        except:
            pass
    
    player_urls = [x for x in player_urls if not 'colleges' in x]
    player_urls = [x for x in player_urls if not 'birthdays' in x]
    
    player_stats_list=[]
    for url in player_urls:
        try:
            player_stats_list.append(nba_2016_player_dict(url))
            print(url)
        except:
            pass
    
    df = pd.DataFrame(player_stats_list)
    df = df[[
        'player_name',
        'position',
        'shooting_hand',
        'height_inches',
        'weight_lbs',
        'college',
        'draft_year',
        'draft_position',
        'season_count',
        'age',
        'current_team',
        'g',
        'gs',
        'mp',
        'pts',
        'fg',
        'fga',
        'fg_perc',
        '3p',
        '3pa',
        '3p_perc',
        '2p',
        '2pa',
        '2p_perc',
        'efg_perc',
        'ft',
        'fta',
        'ft_perc',
        'orb',
        'drb',
        'trb',
        'ast',
        'stl',
        'blk',
        'tov',
        'pf',
        'all_star',
        'per',
        'ws',
        'recent_salary',
        'salary_year',
        'salary_team'
    ]]
    return df


In [10]:
generate_2016_nba_player_df('y')

https://www.basketball-reference.com/players/y/youngja01.html
https://www.basketball-reference.com/players/y/youngjo01.html
https://www.basketball-reference.com/players/y/youngni01.html
https://www.basketball-reference.com/players/y/youngth01.html


,player_name,position,shooting_hand,height_inches,weight_lbs,college,draft_year,draft_position,season_count,age,...,stl,blk,tov,pf,all_star,per,ws,recent_salary,salary_year,salary_team
0,James Young,Shooting Guard,Left,78,215,University of Kentucky,2014,17,3,22,...,0.3,0.1,0.1,0.5,0,10.0,0.3,1825200.0,2016,BOS
1,Joe Young,Point Guard,Right,74,180,University of Oregon,2015,43,2,25,...,0.1,0.0,0.2,0.2,0,11.4,0.1,1052342.0,2016,IND
2,Nick Young,Shooting Guard,Right,79,210,University of Southern California,2007,16,12,32,...,0.6,0.2,0.6,2.3,0,14.1,3.2,5443918.0,2016,LAL
3,Thaddeus Young,Power Forward,Left,80,221,Georgia Institute of Technology,2007,12,12,29,...,1.5,0.4,1.3,1.8,0,14.9,4.6,14153652.0,2016,IND


### Versions utilized for scraping

In [16]:
import sys
import bs4
import re

In [17]:
print('Python version:', sys.version_info)
print('BeautifulSoup version:', bs4.__version__)
print('Pandas version:', pd.__version__)
print('Numpy version:',np.__version__)
print('RegEx version:',re.__version__)

Python version: sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)
BeautifulSoup version: 4.5.3
Pandas version: 0.19.2
Numpy version: 1.12.1
RegEx version: 2.2.1
